In [298]:
#https://www.youtube.com/watch?v=fNxaJsNG3-s&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S

In [299]:
import pandas as pd

In [300]:
clickbait = pd.read_table('data/clickbait_data.txt', header=None)
clickbait

,0
0,Should I Get Bings
1,Which TV Female Friend Group Do You Belong In
2,"The New ""Star Wars: The Force Awakens"" Trailer..."
3,"This Vine Of New York On ""Celebrity Big Brothe..."
4,A Couple Did A Stunning Photo Shoot With Their...
...,...
15994,"There Was A Mini ""Sisterhood Of The Traveling ..."
15995,21 Dogs Who Are Thankful For Their Best Friends
15996,People Are Proving No Dick Is Too Big By Dropp...
15997,"I'm An Atheist, But I'm Not"


In [301]:
non_clickbait = pd.read_table('data/non_clickbait_data.txt', header=None)
non_clickbait

,0
0,Bill Changing Credit Card Rules Is Sent to Oba...
1,"In Hollywood, the Easy-Money Generation Toughe..."
2,1700 runners still unaccounted for in UK's Lak...
3,Yankees Pitchers Trade Fielding Drills for Put...
4,Large earthquake rattles Indonesia; Seventh in...
...,...
15994,Pentagon unable to explain 'mystery missile' v...
15995,Blair: G8 leaders announce $50 billion aid inc...
15996,"To Make Female Hearts Flutter in Iraq, Throw a..."
15997,"British Liberal Democrat Patsy Calton, 56, die..."


In [302]:
clickbait[1]=1
non_clickbait[1]=0
df = pd.merge(clickbait, non_clickbait, how='outer')
df

,0,1
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1
...,...,...
31993,Pentagon unable to explain 'mystery missile' v...,0
31994,Blair: G8 leaders announce $50 billion aid inc...,0
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",0
31996,"British Liberal Democrat Patsy Calton, 56, die...",0


In [303]:
import tensorflow as tf
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [304]:
from sklearn.model_selection import train_test_split
X = df[0].values
y = df[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [305]:
def preprocess_text(text):
    return text.lower()

df[0] = df[0].apply(preprocess_text)
df

,0,1
0,should i get bings,1
1,which tv female friend group do you belong in,1
2,"the new ""star wars: the force awakens"" trailer...",1
3,"this vine of new york on ""celebrity big brothe...",1
4,a couple did a stunning photo shoot with their...,1
...,...,...
31993,pentagon unable to explain 'mystery missile' v...,0
31994,blair: g8 leaders announce $50 billion aid inc...,0
31995,"to make female hearts flutter in iraq, throw a...",0
31996,"british liberal democrat patsy calton, 56, die...",0


In [306]:
vocab_size = 50000
embedding_dim = 16
max_length = 1000
trunc_type='post'
padding_type='post'
oov_token = "<OOV>"

In [307]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [308]:
training_sentences = X_train
testing_sentences = X_test
training_labels = y_train
testing_labels = y_test

In [309]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)
training_labels

array([0, 1, 0, ..., 1, 1, 0], dtype=int64)

In [310]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [311]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 1000, 16)          800000    
                                                                 
 global_average_pooling1d_13  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_26 (Dense)            (None, 24)                408       
                                                                 
 dropout_7 (Dropout)         (None, 24)                0         
                                                                 
 dense_27 (Dense)            (None, 1)                 25        
                                                                 
Total params: 800,433
Trainable params: 800,433
Non-trainable params: 0
_______________________________________________

In [312]:
num_epochs = 10
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing t

In [313]:
sentence = ["I appologize", "They don't make these anymore"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 49ms/step
[[0.50417125]
 [0.9928556 ]]
